Reference:
https://github.com/umbertogriffo/Predictive-Maintenance-using-LSTM/blob/master/src/lstm/regression.py

In [1]:
import keras

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, Activation, Conv1D, MaxPooling1D
%matplotlib inline

In [3]:
jet = pd.read_table('../train_data.txt',sep=' ',header=None)
jet.drop([26,27],axis=1,inplace=True)
jet.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
train_df = jet
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


In [4]:
td = pd.read_table('../test_data.txt',sep=' ',header=None)
td.drop([26,27],axis=1,inplace=True)
td.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
td.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,9.9987,0.2502,100.0,489.05,605.03,1497.17,1304.99,10.52,...,371.69,2388.18,8114.10,8.6476,0.03,369,2319,100.00,28.42,17.1551
1,1,2,20.0026,0.7000,100.0,491.19,607.82,1481.20,1246.11,9.35,...,315.32,2388.12,8053.06,9.2405,0.02,364,2324,100.00,24.29,14.8039
2,1,3,35.0045,0.8400,100.0,449.44,556.00,1359.08,1128.36,5.48,...,183.04,2387.75,8053.04,9.3472,0.02,333,2223,100.00,14.98,8.9125
3,1,4,42.0066,0.8410,100.0,445.00,550.17,1349.69,1127.89,3.91,...,130.40,2387.72,8066.90,9.3961,0.02,332,2212,100.00,10.35,6.4181
4,1,5,24.9985,0.6213,60.0,462.54,536.72,1253.18,1050.69,7.05,...,164.56,2028.05,7865.66,10.8682,0.02,305,1915,84.93,14.31,8.5740


In [5]:
td.groupby('id').max()['cycle'].min()

21

### Kmeans

In [6]:
from sklearn import cluster

In [7]:
k_cluster = cluster.KMeans(n_clusters=6)
k_cluster2 = cluster.KMeans(n_clusters=6) 

In [8]:
def max_min_normalize(df):
    return (df-df.min())/(df.max()-df.min())

In [9]:
feature = jet[['setting1','setting2','setting3']]
test_feature = td[['setting1','setting2','setting3']]

In [10]:
train_data = np.array(max_min_normalize(feature))
test_data = np.array(max_min_normalize(test_feature))

In [11]:
k_cluster.fit(train_data)
k_cluster2.fit(test_data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [12]:
train_df['mode'] = k_cluster.labels_
td['mode'] = k_cluster2.labels_

In [13]:
train_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,mode
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071,1
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665,5
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723,0
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701,5
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53754,260,312,20.0037,0.7000,100.0,491.19,608.79,1495.60,1269.51,9.35,...,2389.02,8169.64,9.3035,0.03,369,2324,100.00,24.36,14.5189,3
53755,260,313,10.0022,0.2510,100.0,489.05,605.81,1514.32,1324.12,10.52,...,2388.42,8245.36,8.7586,0.03,374,2319,100.00,28.10,16.9454,4
53756,260,314,25.0041,0.6200,60.0,462.54,537.48,1276.24,1057.92,7.05,...,2030.33,7971.25,11.0657,0.02,310,1915,84.93,14.19,8.5503,0
53757,260,315,25.0033,0.6220,60.0,462.54,537.84,1272.95,1066.30,7.05,...,2030.35,7972.47,11.0537,0.02,311,1915,84.93,14.05,8.3729,0


In [14]:
td

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,mode
0,1,1,9.9987,0.2502,100.0,489.05,605.03,1497.17,1304.99,10.52,...,2388.18,8114.10,8.6476,0.03,369,2319,100.00,28.42,17.1551,4
1,1,2,20.0026,0.7000,100.0,491.19,607.82,1481.20,1246.11,9.35,...,2388.12,8053.06,9.2405,0.02,364,2324,100.00,24.29,14.8039,3
2,1,3,35.0045,0.8400,100.0,449.44,556.00,1359.08,1128.36,5.48,...,2387.75,8053.04,9.3472,0.02,333,2223,100.00,14.98,8.9125,5
3,1,4,42.0066,0.8410,100.0,445.00,550.17,1349.69,1127.89,3.91,...,2387.72,8066.90,9.3961,0.02,332,2212,100.00,10.35,6.4181,0
4,1,5,24.9985,0.6213,60.0,462.54,536.72,1253.18,1050.69,7.05,...,2028.05,7865.66,10.8682,0.02,305,1915,84.93,14.31,8.5740,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33986,259,119,35.0015,0.8403,100.0,449.44,555.56,1366.01,1129.47,5.48,...,2388.39,8088.36,9.3215,0.02,334,2223,100.00,14.94,8.9065,5
33987,259,120,42.0066,0.8405,100.0,445.00,549.42,1351.13,1123.86,3.91,...,2388.31,8108.48,9.3542,0.02,332,2212,100.00,10.57,6.4075,0
33988,259,121,42.0061,0.8400,100.0,445.00,549.65,1349.14,1118.91,3.91,...,2388.34,8098.77,9.3836,0.02,331,2212,100.00,10.57,6.4805,0
33989,259,122,0.0024,0.0003,100.0,518.67,642.58,1589.61,1408.16,14.62,...,2388.00,8161.85,8.4279,0.03,393,2388,100.00,39.08,23.3589,1


### 数据处理

In [15]:
# Data Labeling - generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,mode,RUL
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071,1,148
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665,5,147
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723,0,146
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701,5,145
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286,0,144


In [16]:
# normalize the columns in the training data.
# MinMax normalization
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,mode,RUL,cycle_norm
0,1,1,0.833134,0.997625,1.0,0.060269,0.181576,0.311201,0.273095,0.146592,...,0.369947,0.0,0.322917,0.651163,1.0,0.156036,0.159082,0.2,148,0.000000
1,1,2,0.999767,0.998575,1.0,0.000000,0.131847,0.296600,0.245535,0.000000,...,0.381407,0.0,0.281250,0.627907,1.0,0.007888,0.014562,1.0,147,0.002653
2,1,3,0.595096,0.738480,0.0,0.238089,0.016332,0.035297,0.056997,0.293184,...,0.936731,0.0,0.062500,0.000000,0.0,0.133745,0.151414,0.0,146,0.005305
3,1,4,0.999993,0.999525,1.0,0.000000,0.128269,0.298795,0.246979,0.000000,...,0.372400,0.0,0.270833,0.627907,1.0,0.014060,0.026144,1.0,145,0.007958
4,1,5,0.595137,0.736698,0.0,0.238089,0.014130,0.037871,0.058152,0.293184,...,0.937537,0.0,0.062500,0.000000,0.0,0.135460,0.143240,0.0,144,0.010610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53754,260,312,0.476188,0.831354,1.0,0.626985,0.672172,0.682297,0.591489,0.507937,...,0.354350,1.0,0.687500,0.864693,1.0,0.486283,0.483993,0.6,4,0.824934
53755,260,313,0.238102,0.298100,1.0,0.597937,0.644830,0.733008,0.722934,0.617180,...,0.154840,1.0,0.739583,0.854123,1.0,0.614540,0.622022,0.8,3,0.827586
53756,260,314,0.595222,0.736342,0.0,0.238089,0.017892,0.088067,0.082198,0.293184,...,0.999561,0.0,0.072917,0.000000,0.0,0.137517,0.144474,0.0,2,0.830239
53757,260,315,0.595203,0.738717,0.0,0.238089,0.021195,0.079155,0.102368,0.293184,...,0.995167,0.0,0.083333,0.000000,0.0,0.132716,0.134383,0.0,1,0.832891


In [17]:
# normalize the columns in the test data.
# MinMax normalization
td['cycle_norm'] = td['cycle']
cols_normalize = td.columns.difference(['id','cycle'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(td[cols_normalize]), 
                             columns=cols_normalize, 
                             index=td.index)
join_df = td[td.columns.difference(cols_normalize)].join(norm_train_df)
td = join_df.reindex(columns = td.columns)
td

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,mode,cycle_norm
0,1,1,0.238019,0.297150,1.0,0.597937,0.639243,0.691329,0.680395,0.617180,...,0.670742,0.117564,1.0,0.694737,0.854123,1.0,0.625944,0.630724,0.8,0.000000
1,1,2,0.476162,0.831354,1.0,0.626985,0.664861,0.647519,0.534663,0.507937,...,0.514944,0.337173,0.0,0.642105,0.864693,1.0,0.484214,0.496301,0.6,0.002732
2,1,3,0.833282,0.997625,1.0,0.060269,0.189055,0.312512,0.243225,0.146592,...,0.514893,0.376695,0.0,0.315789,0.651163,1.0,0.164722,0.159476,1.0,0.005464
3,1,4,0.999967,0.998812,1.0,0.000000,0.135525,0.286753,0.242061,0.000000,...,0.550269,0.394807,0.0,0.305263,0.627907,1.0,0.005834,0.016866,0.0,0.008197
4,1,5,0.595089,0.737886,0.0,0.238089,0.012028,0.022001,0.050986,0.293184,...,0.036627,0.940070,0.0,0.021053,0.000000,0.0,0.141730,0.140123,0.4,0.010929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33986,259,119,0.833210,0.997981,1.0,0.060269,0.185015,0.331523,0.245972,0.146592,...,0.605044,0.367175,0.0,0.326316,0.651163,1.0,0.163349,0.159133,1.0,0.322404
33987,259,120,0.999967,0.998219,1.0,0.000000,0.128638,0.290703,0.232087,0.000000,...,0.656398,0.379287,0.0,0.305263,0.627907,1.0,0.013384,0.016260,0.0,0.325137
33988,259,121,0.999955,0.997625,1.0,0.000000,0.130750,0.285244,0.219835,0.000000,...,0.631614,0.390177,0.0,0.294737,0.627907,1.0,0.013384,0.020433,0.0,0.327869
33989,259,122,0.000057,0.000356,1.0,1.000000,0.984024,0.944915,0.935747,1.000000,...,0.792618,0.036188,1.0,0.947368,1.000000,1.0,0.991764,0.985410,0.2,0.330601


Keras LSTM layers expect an input in the shape of a numpy array of 3 dimensions (samples, time steps, features) where samples is the number of training sequences, time steps is the look back window or sequence length and features is the number of features of each sequence at each time step.

In [18]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,112),(50,192)
    # 0 50 -> from row 0 to row 50
    # 1 51 -> from row 1 to row 51
    # 2 52 -> from row 2 to row 52
    # ...
    # 111 191 -> from row 111 to 191

In [19]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3','cycle_norm','mode']
# sequence_cols = ['setting1', 'setting2', 'setting3','cycle_norm']
sequence_cols.extend(sensor_cols)

In [20]:
sequence_length = 20

In [21]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(48559, 20, 26)

In [22]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [23]:
# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape
label_array

array([[128.],
       [127.],
       [126.],
       ...,
       [  2.],
       [  1.],
       [  0.]], dtype=float32)

### LSTM Network
Next, we build a deep network. The first layer is an LSTM layer with 100 units followed by another LSTM layer with 50 units. Dropout is also applied after each LSTM layer to control overfitting. Final layer is a Dense output layer with single unit and sigmoid activation since this is a binary classification problem.

### USE the model in github

In [24]:
import keras.backend as K

In [33]:
def r2_keras(y_true, y_pred):
    """Coefficient of Determination 
    """
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [34]:
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()
model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=nb_out))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae',r2_keras])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 20, 100)           50800     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 100)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 81,051
Trainable params: 81,051
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
# fit the network
model_path='4reg.h5'
history = model.fit(seq_array, label_array, epochs=100, batch_size=200, validation_split=0.05, verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
          )

# list all data in history
print(history.history.keys())

NameError: name 'model' is not defined

In [35]:
model.load_weights('4reg.h5')
ans = model.predict(seq_array,verbose=1)
ans

48559/48559 [==============================] - 27s 565us/step


array([[179.98935 ],
       [178.33348 ],
       [159.76498 ],
       ...,
       [ 12.370204],
       [ 12.016957],
       [ 13.830347]], dtype=float32)

In [39]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Loss: {}'.format(scores[0]))

48559/48559 [==============================] - 13s 267us/step
Loss: 1384.0967982289878


### Calculate result

In [36]:
# We pick the last sequence for each id in the test data
test = [td[td['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in td['id'].unique() if len(td[td['id']==id]) >= sequence_length]

test = np.asarray(test).astype(np.float32)

In [38]:
result = model.predict(test,verbose=1)
result

259/259 [==============================] - 0s 596us/step


array([[ 11.151736 ],
       [149.32538  ],
       [ 83.99304  ],
       [110.665    ],
       [ 16.787735 ],
       [148.4408   ],
       [ 15.351172 ],
       [ 36.26513  ],
       [  8.919941 ],
       [137.01791  ],
       [ 27.952438 ],
       [123.181404 ],
       [ 65.00417  ],
       [  7.299381 ],
       [ 78.21964  ],
       [ 62.30776  ],
       [ 58.372677 ],
       [146.38222  ],
       [ 23.510666 ],
       [ 90.42477  ],
       [ 18.411762 ],
       [ 13.414752 ],
       [146.5156   ],
       [ 86.06526  ],
       [ 71.485985 ],
       [121.19422  ],
       [144.36896  ],
       [172.76967  ],
       [ 87.05645  ],
       [ 61.078037 ],
       [100.587456 ],
       [ 12.450218 ],
       [ 50.081215 ],
       [139.47684  ],
       [109.716606 ],
       [ 70.6876   ],
       [115.29105  ],
       [ 37.266308 ],
       [ 41.961452 ],
       [ 29.306286 ],
       [ 36.114506 ],
       [ 27.55508  ],
       [ 71.12164  ],
       [ 18.861294 ],
       [  9.326402 ],
       [ 5

In [40]:
result.shape

(259, 1)

---